# Программирование на языке Python
## Семинар 9. Pandas

Данные взяты [отсюда](https://github.com/datasets/covid-19).

In [1]:
import pandas as pd
import numpy as np

#### Задание 1
Импортируйте и проведите первичное изучение данных (соответствующим типу каждой переменной).

In [ ]:
# ЗАМЕТКИ
# 1 переменная Province/State имеет пропущенные значения
# 2 Date, возможно, имеет смысл перевести в datetime -> необходимо
# 3 Recovered имеет тип float, хотя должно бы int -> это из-за пропущенных значений +++++++
# 4 следует стандартизировать названия переменных
# 5 связаны ли между собой пропущенные значения в переменных?
# 6 в числовых переменных есть экстремально большие значения
# 7 некоторые страны встречаются больше раз, чем подавляющее большинство стран

In [2]:
# анализ таблицы в целом
covid = pd.read_csv('time-series-19-covid-combined.csv')
covid.head()

,Date,Country/Region,Province/State,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,NaN,0,0.0,0
1,2020-01-23,Afghanistan,NaN,0,0.0,0
2,2020-01-24,Afghanistan,NaN,0,0.0,0
3,2020-01-25,Afghanistan,NaN,0,0.0,0
4,2020-01-26,Afghanistan,NaN,0,0.0,0


In [8]:
covid.sample(5)

,Date,Country/Region,Province/State,Confirmed,Recovered,Deaths
183923,2020-12-10,Sao Tome and Principe,NaN,1009,946.0,17
18848,2020-04-11,Belarus,NaN,2226,172.0,23
148441,2022-02-05,Marshall Islands,NaN,7,0.0,0
152636,2020-03-06,Monaco,NaN,1,0.0,0
175092,2021-05-04,Poland,NaN,2808052,2536931.0,68133


In [10]:
np.array([1, 2, 3, np.nan])

array([ 1.,  2.,  3., nan])

In [9]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231744 entries, 0 to 231743
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Date            231744 non-null  object 
 1   Country/Region  231744 non-null  object 
 2   Province/State  72624 non-null   object 
 3   Confirmed       231744 non-null  int64  
 4   Recovered       218688 non-null  float64
 5   Deaths          231744 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 10.6+ MB


In [3]:
# пункт 4
covid_renamed = covid.rename(columns=lambda name: name.lower().replace('/', '_'))

In [4]:
# рассмотрим переменные
# вначале числовые
numeric_cols = ['confirmed', 'recovered', 'deaths']
covid_renamed[numeric_cols].describe().round(2)

,confirmed,recovered,deaths
count,231744.00,218688.00,231744.00
mean,513236.17,106211.62,9760.17
std,3002239.15,839741.31,49749.87
min,0.00,0.00,0.00
25%,259.00,0.00,2.00
50%,5725.00,55.00,73.00
75%,109564.50,5002.00,1777.00
max,80625120.00,30974748.00,988609.00


In [5]:
# теперь категориальные
vc_countries = covid_renamed['country_region'].value_counts()
vc_countries

country_region
China             27744
Canada            13056
United Kingdom    11424
France             9792
Australia          6528
                  ...  
Guinea              816
Guinea-Bissau       816
Guyana              816
Haiti               816
Zimbabwe            816
Name: count, Length: 198, dtype: int64

In [22]:
covid_renamed['country_region']

0         Afghanistan
1         Afghanistan
2         Afghanistan
3         Afghanistan
4         Afghanistan
             ...     
231739       Zimbabwe
231740       Zimbabwe
231741       Zimbabwe
231742       Zimbabwe
231743       Zimbabwe
Name: country_region, Length: 231744, dtype: object

In [21]:
vc_countries.value_counts()

count
816      190
27744      1
13056      1
11424      1
9792       1
6528       1
4080       1
2448       1
1632       1
Name: count, dtype: int64

#### Задание 2
- переименуйте переменные, приведя их к более "стандартному" и удобному виду;
- проверьте данные на корректность: одинаково ли представлена история для всех стран? Есть ли пропущенные значения? Сколько их? По какому принципу они пропущены? Если ответ на эти вопросы вскроет какие-либо несоответствия в данных, предложите причины и варианты решения несоответствий;
- для всех ли стран представлены одни и те же даты? Отсортируйте датасет по стране и дате, чтобы последовательность была строго упорядочена во времени.

In [ ]:
# ЗАМЕТКИ
# 1 переменная Province/State имеет пропущенные значения +++++++
# 2 Date, возможно, имеет смысл перевести в datetime -> необходимо +++++++
# 3 Recovered имеет тип float, хотя должно бы int -> это из-за пропущенных значений +++++++
# 4 следует стандартизировать названия переменных +++++++
# 5 связаны ли между собой пропущенные значения в переменных? +++++++
# 6 в числовых переменных есть экстремально большие значения
# 7 некоторые страны встречаются больше раз, чем подавляющее большинство стран -> для некоторых стран есть данные по регионам +++++++

# 8 необходимо агрегировать значения числовых переменных по странам из пункта 7

In [6]:
# пункт 2
covid_renamed['date_dt'] = pd.to_datetime(covid_renamed['date'], format='%Y-%m-%d')

In [24]:
covid_renamed['date_dt']

0        2020-01-22
1        2020-01-23
2        2020-01-24
3        2020-01-25
4        2020-01-26
            ...    
231739   2022-04-12
231740   2022-04-13
231741   2022-04-14
231742   2022-04-15
231743   2022-04-16
Name: date_dt, Length: 231744, dtype: datetime64[ns]

In [27]:
# пункт 1, 5, 7
covid_renamed[covid_renamed['province_state'].notna()]

,date,country_region,province_state,confirmed,recovered,deaths,date_dt
7344,2020-01-22,Australia,Australian Capital Territory,0,0.0,0,2020-01-22
7345,2020-01-23,Australia,Australian Capital Territory,0,0.0,0,2020-01-23
7346,2020-01-24,Australia,Australian Capital Territory,0,0.0,0,2020-01-24
7347,2020-01-25,Australia,Australian Capital Territory,0,0.0,0,2020-01-25
7348,2020-01-26,Australia,Australian Capital Territory,0,0.0,0,2020-01-26
...,...,...,...,...,...,...,...
222763,2022-04-12,United Kingdom,Turks and Caicos Islands,5923,0.0,36,2022-04-12
222764,2022-04-13,United Kingdom,Turks and Caicos Islands,5936,0.0,36,2022-04-13
222765,2022-04-14,United Kingdom,Turks and Caicos Islands,5936,0.0,36,2022-04-14
222766,2022-04-15,United Kingdom,Turks and Caicos Islands,5936,0.0,36,2022-04-15


In [35]:
# все страны с непропущенным значением в province_state
unusual_countries1 = set(covid_renamed.loc[covid_renamed['province_state'].notna(), 'country_region'].unique())

In [36]:
# все страны с аномальным количеством строк
unusual_countries2 = set(vc_countries.index[vc_countries.values != 816])

In [37]:
unusual_countries1.symmetric_difference(unusual_countries2)

set()

In [39]:
# recovered
(covid_renamed['province_state'].isna() & covid_renamed['recovered'].isna()).any()

False

In [40]:
covid_renamed[covid_renamed['recovered'].isna()]

,date,country_region,province_state,confirmed,recovered,deaths,date_dt
32640,2020-01-22,Canada,Alberta,0,NaN,0,2020-01-22
32641,2020-01-23,Canada,Alberta,0,NaN,0,2020-01-23
32642,2020-01-24,Canada,Alberta,0,NaN,0,2020-01-24
32643,2020-01-25,Canada,Alberta,0,NaN,0,2020-01-25
32644,2020-01-26,Canada,Alberta,0,NaN,0,2020-01-26
...,...,...,...,...,...,...,...
45691,2022-04-12,Canada,Yukon,3940,NaN,24,2022-04-12
45692,2022-04-13,Canada,Yukon,4000,NaN,24,2022-04-13
45693,2022-04-14,Canada,Yukon,4015,NaN,24,2022-04-14
45694,2022-04-15,Canada,Yukon,4015,NaN,24,2022-04-15


In [42]:
covid_renamed.loc[covid_renamed['recovered'].isna(), 'country_region'].unique()

array(['Canada'], dtype=object)

#### Задание 3

Перед тем, как выполнять данное задание, подумайте, не лучше ли будет каким-либо образом поменять существующие показатели пандемии.

- как меняется соотношения типа обнаруженные / умершие и т. д. во времени (в среднемировом разрезе)?
- если смотреть в среднемировом разрезе, как можно оценить, на какой день болезни шанс заразить кого-нибудь больше всего? Попробуйте проверить, можно ли говорить о какой-либо статистической значимости в контексте вашего ответа;
- если смотреть в среднемировом разрезе, как можно оценить, на какой день болезни шанс умереть больше всего? Попробуйте проверить, можно ли говорить о какой-либо статистической значимости в контексте вашего ответа.

In [43]:
covid_renamed.head()

,date,country_region,province_state,confirmed,recovered,deaths,date_dt
0,2020-01-22,Afghanistan,NaN,0,0.0,0,2020-01-22
1,2020-01-23,Afghanistan,NaN,0,0.0,0,2020-01-23
2,2020-01-24,Afghanistan,NaN,0,0.0,0,2020-01-24
3,2020-01-25,Afghanistan,NaN,0,0.0,0,2020-01-25
4,2020-01-26,Afghanistan,NaN,0,0.0,0,2020-01-26


In [ ]:
# ЗАМЕТКИ
# 6 в числовых переменных есть экстремально большие значения
# 8 необходимо агрегировать значения числовых переменных по странам из пункта 7 ++++++

In [ ]:
# пункт 8
# уникальные ключи: [date, country_region, province_state] -> [date, country_region]
# group по [date, country_region], aggregate using SUM

In [65]:
grouped = covid_renamed[covid_renamed['country_region'].isin(unusual_countries1)].groupby(['country_region', 'date'], as_index=False)

In [66]:
grouped_list = list(grouped)

In [67]:
grouped_list[0]

(('Australia', '2020-01-22'),
              date country_region                province_state  confirmed  \
 7344   2020-01-22      Australia  Australian Capital Territory          0   
 8160   2020-01-22      Australia               New South Wales          0   
 8976   2020-01-22      Australia            Northern Territory          0   
 9792   2020-01-22      Australia                    Queensland          0   
 10608  2020-01-22      Australia               South Australia          0   
 11424  2020-01-22      Australia                      Tasmania          0   
 12240  2020-01-22      Australia                      Victoria          0   
 13056  2020-01-22      Australia             Western Australia          0   
 
        recovered  deaths    date_dt  
 7344         0.0       0 2020-01-22  
 8160         0.0       0 2020-01-22  
 8976         0.0       0 2020-01-22  
 9792         0.0       0 2020-01-22  
 10608        0.0       0 2020-01-22  
 11424        0.0       0 2020-0

In [59]:
grouped_list[0][0]

('Afghanistan',)

In [60]:
grouped_list[0][1]

,date,country_region,province_state,confirmed,recovered,deaths,date_dt
0,2020-01-22,Afghanistan,NaN,0,0.0,0,2020-01-22
1,2020-01-23,Afghanistan,NaN,0,0.0,0,2020-01-23
2,2020-01-24,Afghanistan,NaN,0,0.0,0,2020-01-24
3,2020-01-25,Afghanistan,NaN,0,0.0,0,2020-01-25
4,2020-01-26,Afghanistan,NaN,0,0.0,0,2020-01-26
...,...,...,...,...,...,...,...
811,2022-04-12,Afghanistan,NaN,178257,0.0,7676,2022-04-12
812,2022-04-13,Afghanistan,NaN,178295,0.0,7676,2022-04-13
813,2022-04-14,Afghanistan,NaN,178352,0.0,7676,2022-04-14
814,2022-04-15,Afghanistan,NaN,178373,0.0,7676,2022-04-15


In [53]:
%%timeit
# вариант 1
covid_agg = covid_renamed.groupby(['date', 'date_dt', 'country_region'], as_index=False)[numeric_cols].sum()  # .agg()
# covid_agg.head()

33.9 ms ± 687 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [54]:
%%timeit
# вариант 2
covid_to_agg = covid_renamed[covid_renamed['country_region'].isin(unusual_countries1)]
covid_notto_agg = covid_renamed[~covid_renamed['country_region'].isin(unusual_countries1)]

covid_to_agg_agg = covid_to_agg.groupby(['date', 'date_dt', 'country_region'], as_index=False)[numeric_cols].sum()

covid_agg2 = pd.concat([covid_to_agg_agg, covid_notto_agg.drop(columns='province_state')], ignore_index=True)

21.3 ms ± 789 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
covid_agg2

,date,date_dt,country_region,confirmed,recovered,deaths
0,2020-01-22,2020-01-22,Australia,0,0.0,0
1,2020-01-22,2020-01-22,Canada,0,0.0,0
2,2020-01-22,2020-01-22,China,548,28.0,17
3,2020-01-22,2020-01-22,Denmark,0,0.0,0
4,2020-01-22,2020-01-22,France,0,0.0,0
...,...,...,...,...,...,...
161563,2022-04-12,2022-04-12,Zimbabwe,247094,0.0,5460
161564,2022-04-13,2022-04-13,Zimbabwe,247160,0.0,5460
161565,2022-04-14,2022-04-14,Zimbabwe,247208,0.0,5462
161566,2022-04-15,2022-04-15,Zimbabwe,247237,0.0,5462


In [69]:
# как меняется соотношения типа обнаруженные / умершие и т. д. во времени (в среднемировом разрезе)?
covid_world = covid_agg.groupby(['date', 'date_dt'], as_index=False)[numeric_cols].sum()
covid_world.head()

,date,date_dt,confirmed,recovered,deaths
0,2020-01-22,2020-01-22,557,30.0,17
1,2020-01-23,2020-01-23,657,32.0,18
2,2020-01-24,2020-01-24,944,39.0,26
3,2020-01-25,2020-01-25,1437,42.0,42
4,2020-01-26,2020-01-26,2120,56.0,56


#### Задание 4

Изобразите распределение всех трех показателей прироста с помощью:

1. Гистограммы
2. Ящика с усами (boxplot)
3. Функции распределения

На одном графике.

Повторите то же самое для логарифмированных показателей.

In [ ]:
# наш код здесь

#### Задание 5

Изобразите для шести понравившихся вам стран взаимосвязь между логарифмированными показателями прироста заболевших и умерших (по графику на страну, все на одной картинке). Сделайте это тремя способами: с помощью `matplotlib`, `seaborn` и `plotly`.

In [ ]:
# наш код здесь